Source: https://www.kaggle.com/c/word2vec-nlp-tutorial

In [46]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

### Reading the Data

In [20]:
train = pd.read_csv('word2vec-nlp-tutorial/labeledTrainData.tsv',\
                    delimiter='\t', header=0, quoting=3)
train.shape

(25000, 3)

In [21]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [22]:
train.review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

### Data Cleaning and Text Preprocessing

In [23]:
example1 = BeautifulSoup(train.review[0], 'lxml')
example1.get_text()

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [25]:
letters_only = re.sub(string=example1.get_text(), pattern='[^a-zA-Z]', repl=' ')
# [] indicates group membership and ^ means "not". 
# In other words, the re.sub() statement above says, 
# "Find anything that is NOT a lowercase letter (a-z) or an upper case letter (A-Z), 
# and replace it with a space."
letters_only

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [26]:
lower_case = letters_only.lower()
words = lower_case.split()

In [29]:
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words('english')]
words

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

In [35]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
    # Remove HTML
    review = BeautifulSoup(raw_review, 'lxml').get_text()
    
    # Remove non-letters
    review = re.sub(string=review, pattern='[^a-zA-Z]', repl=' ')
    
    # Convert to lower case, split into individual words
    review = review.lower().split()
    
    # In Python, searching a set is much faster than searching
    # a list, so convert the stop words to a set
    stops = stopwords.words('english')
    
    # Remove stop words
    review = [w for w in review if not w in stops]
    
    # Join the words back into one string separated by space, 
    # and return the result.
    review = ' '.join(review)
    
    return review

In [36]:
review_to_words(train.review[0])

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [38]:
print("Cleaning and parsing the training set movie reviews...\n")

clean_train_reviews = []

for i in range(train.review.size):
    clean_train_reviews.append(review_to_words(train.review[i]))
    if (i+1)%5000 == 0:
        print('Cleaned {} out of {} Reviews.\n'.format(i+1, train.review.size))

Cleaning and parsing the training set movie reviews...

Cleaned 5000 out of 25000 Reviews.

Cleaned 10000 out of 25000 Reviews.

Cleaned 15000 out of 25000 Reviews.

Cleaned 20000 out of 25000 Reviews.

Cleaned 25000 out of 25000 Reviews.



### Creating Features from a Bag of Words (Using scikit-learn)

In [42]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000, \
                             preprocessor=None, stop_words=None, \
                             tokenizer=None)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()
train_data_features.shape

(25000, 5000)

In [44]:
vocab = vectorizer.get_feature_names()
vocab;

### Modeling

In [48]:
print('Training the random forest...')

clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
clf.fit(train_data_features, train.sentiment)

Training the random forest...


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Creating a Submission

In [51]:
test = pd.read_csv('word2vec-nlp-tutorial/testData.tsv',\
                    delimiter='\t', header=0, quoting=3)
test.shape

(25000, 2)

In [53]:
clean_test_reviews = []

for i in range(len(test.review)):
    clean_test_reviews.append(review_to_words(test.review[i]))
    if (i+1)%5000==0:
        print('Cleaned {} out of {} Reviews.\n'.format(i+1, len(test.review)))

Cleaned 5000 out of 25000 Reviews.

Cleaned 10000 out of 25000 Reviews.

Cleaned 15000 out of 25000 Reviews.

Cleaned 20000 out of 25000 Reviews.

Cleaned 25000 out of 25000 Reviews.



In [54]:
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [55]:
pred = clf.predict(test_data_features)

In [56]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':pred})
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )